# Collapsing Things in netCDF4
# May 19, 2017

In [98]:
import numpy as np
import numpy.ma as ma
import netCDF4 as nc
import matplotlib.pyplot as plt
%matplotlib inline 
from salishsea_tools import nc_tools
import glob

In [99]:
mesh_mask = nc.Dataset('/home/vdo/MEOPAR/NEMO-forcing/grid/mesh_mask201702.nc')    

test_u = glob.glob('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h*U*')
test_v = glob.glob('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h*V*')
test_w = glob.glob('/home/vdo/MEOPAR/completed-runs/VAT19/SalishSea_1h*W*')

In [1]:
def pcourantu(files,meshmask):
    """Given a list of U filenames and a mesh mask, returns an array with the unscaled Courant numbers.
    
    :arg files: list of U filenames
    
    :arg meshmask: mesh mask 
    :type meshmask: :py:class:'netCDF4.Dataset'
    
    :returns: Numpy MaskedArray with unscaled Courant numbers.
    :rtype: :py:class: 'numpy.ma.core.MaskedArray'
    """
    
    delta_x = meshmask['e1u']
    with nc_tools.scDataset(files) as f:   #merging files
        nt,nz,ny,nx = f.variables['vozocrtx'].shape
        ubdx = np.zeros((nz,ny,nx))
        for n in range (nt):
            u = np.abs(f.variables['vozocrtx'][n,:,:,:] / delta_x)
            ubdx = np.maximum(u,ubdx)    #taking maximum over time
        new_u = np.zeros((ny,nx))
        for m in range(0,nz):
            u = ubdx[m,:,:]
            new_u = np.maximum(u,new_u)  #taking maximum over deptht
            
    return new_u

In [101]:
pcourantu(test_u,mesh_mask)

masked_array(data =
 [[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]],
             mask =
 [[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]],
       fill_value = 1e+20)

In [2]:
def pcourantv(files,meshmask):
    """Given a list of V filenames and a mesh mask, returns an array with the unscaled Courant numbers.
    
    :arg files: list of V filenames
    
    :arg meshmask: mesh mask 
    :type meshmask: :py:class:'netCDF4.Dataset'
    
    :returns: Numpy MaskedArray with unscaled Courant numbers.
    :rtype: :py:class: 'numpy.ma.core.MaskedArray'
    """
    
    delta_y = meshmask['e2v']
    with nc_tools.scDataset(files) as f:    #merging files
        nt,nz,ny,nx = f.variables['vomecrty'].shape
        vbdx = np.zeros((nz,ny,nx))
        for n in range (nt):
            v = np.abs(f.variables['vomecrty'][n,:,:,:] / delta_y)
            vbdx = np.maximum(v,vbdx)   #taking maximum over time
        new_v = np.zeros((ny,nx))
        for m in range(0,nz):
            v = vbdx[m,:,:]
            new_v = np.maximum(v,new_v)   #taking maximum over deptht
            
    return new_v

In [107]:
pcourantv(test_v,mesh_mask)

masked_array(data =
 [[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]],
             mask =
 [[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]],
       fill_value = 1e+20)

In [3]:
def pcourantw(files,meshmask):
    """Given a list of W filenames and a mesh mask, returns an array with the unscaled Courant numbers.
    
    :arg files: list of W filenames
    
    :arg meshmask: mesh mask 
    :type meshmask: :py:class:'netCDF4.Dataset'
    
    :returns: Numpy MaskedArray with unscaled Courant numbers.
    :rtype: :py:class: 'numpy.ma.core.MaskedArray'
    """
    
    with nc_tools.scDataset(files) as f:    #merging files
        nt,nz,ny,nx = f.variables['vovecrtz'].shape
        wbdx = np.zeros((nz,ny,nx))
        delta_z = meshmask['e3w_1d']
        new_z1 = np.expand_dims(delta_z[:],axis=2)
        new_z2 = np.swapaxes(new_z1,0,1)
        ones = np.ones((nz,ny,nx))
        new_z3 = ones*new_z2
        for n in range (nt):
            w = np.abs(f.variables['vovecrtz'][n,:,:,:] / new_z3)
            wbdx = np.maximum(w,wbdx)    #taking maximum over time
        new_w = np.zeros((ny,nx))
        for m in range(0,nz):
            w = wbdx[m,:,:]
            new_w = np.maximum(w,new_w)  #taking maximum over deptht
            
    return new_w

In [109]:
pcourantw(test_w,mesh_mask)

masked_array(data =
 [[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]
 [0.0 0.0 0.0 ..., -- -- --]],
             mask =
 [[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]
 [False False False ...,  True  True  True]],
       fill_value = 1e+20)